In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import asyncio
import aiohttp
import time

In [7]:
doktorlar = pd.read_csv("doktorlar_id.csv")

doktorlar.tail()

,id,image_link,title,name,is_verified,profile,gender,location,speciality
19947,19948,https://www.doktorsitesi.com/media/cache/profi...,Dyt.,Almina Çöğürcü,NaN,https://www.doktorsitesi.com/almina-cogurcu/di...,female,antalya,diyetisyen
19948,19949,https://www.doktorsitesi.com/media/cache/profi...,Psk. Dan.,Alperen Kırkpulat,NaN,https://www.doktorsitesi.com/alperen-kirkpulat...,female,izmir,psikolojik-danisman
19949,19950,https://www.doktorsitesi.com/media/cache/profi...,Aile Danışmanı,Ayşe Bozkurt,NaN,https://www.doktorsitesi.com/ayse-bozkurt/aile...,female,gaziantep,aile-danismani
19950,19951,https://www.doktorsitesi.com/media/cache/profi...,Psk. Dan.,Ayşen Özdoğancı,NaN,https://www.doktorsitesi.com/aysen-ozdoganci/p...,female,isparta,psikolojik-danisman
19951,19952,https://www.doktorsitesi.com/media/cache/profi...,Dil ve Konuşma Terapisti,Aynur İkra Demir,NaN,https://www.doktorsitesi.com/aynur-ikra-demir/...,female,istanbul,dil-konusma-bozuklari-b


In [8]:
# print the first 5 rows profile links
for i in range(9941, 9952):
    print(doktorlar["profile"][i])

https://www.doktorsitesi.com/dr-ayse-irem-akyuz/pratisyen-hekimlik/sanliurfa
https://www.doktorsitesi.com/dr-ayse-akyildiz/pratisyen-hekimlik/sanliurfa
https://www.doktorsitesi.com/dr-aysegul-aydogan/pratisyen-hekimlik/sanliurfa
https://www.doktorsitesi.com/dr-aysegul-uzen/pratisyen-hekimlik/sanliurfa
https://www.doktorsitesi.com/dr-aziz-demirbas/pratisyen-hekimlik/sanliurfa
https://www.doktorsitesi.com/op-dr-aziz-dinek/kulak-burun-bogaz-hastaliklari/balikesir
https://www.doktorsitesi.com/dr-abdul-bozdag/pratisyen-hekimlik/usak
https://www.doktorsitesi.com/dr-ahmet-serhan-sunbul/pratisyen-hekimlik/usak
https://www.doktorsitesi.com/dr-ahmet-ayan/pratisyen-hekimlik/usak
https://www.doktorsitesi.com/dr-ahmet-mart/pratisyen-hekimlik/usak
https://www.doktorsitesi.com/dr-alparslan-ozdemir/pratisyen-hekimlik/usak


In [9]:
error_indices = []

In [32]:
import json

# get the details of the profile
async def get_profile_details(url, session, i):
    try:
        async with session.get(url) as response:
            html = await response.text()
            soup = BeautifulSoup(html, "html.parser")

            doctor_details = {}

            expert_profile_header = soup.find("div", class_="expert-profile-header")
            doctor_details['data_id'] = expert_profile_header["data-id"]

            expert_point = expert_profile_header.find("div", class_="expert-point")
            if expert_point:
                doctor_details['expert_point'] = expert_point.text.strip()
            else:
                doctor_details['expert_point'] = None

            services = expert_profile_header.find_all("a", class_="service-list-item")
            doctor_details['services'] = json.dumps([service.text.strip() for service in services])

            about_modal = soup.find("div", id="aboutModal")
            about_content = about_modal.find("div", id="aboutContent")
            doctor_details['about_content'] = about_content.text.strip()

            eed_items = soup.find_all("div", class_="eed-item")
            doctor_details['eed_items'] = json.dumps([item.text.strip() for item in eed_items])

            ds_expert_interests = soup.find("div", class_="ds-expert-interest")
            expert_interests = ds_expert_interests.find_all("a")
            doctor_details['expert_interests'] = json.dumps([interest.text.strip() for interest in expert_interests])

            experience_company = soup.find("div", class_="experience-company")
            if experience_company:
                doctor_details['experience_company'] = experience_company.text.strip()
            else:
                doctor_details['experience_company'] = None

            # get inside this tag too <script type="application/ld+json">

            application_ld_json = soup.find("script", type="application/ld+json")
            doctor_details['application_ld_json'] = application_ld_json.text.strip()

            # add the details to the doktorlar dataframe
            doktorlar.loc[i, 'data_id'] = doctor_details['data_id']
            doktorlar.loc[i, 'expert_point'] = doctor_details['expert_point']
            doktorlar.loc[i, 'services'] = doctor_details['services']
            doktorlar.loc[i, 'about_content'] = doctor_details['about_content']
            doktorlar.loc[i, 'eed_items'] = doctor_details['eed_items']
            doktorlar.loc[i, 'expert_interests'] = doctor_details['expert_interests']
            doktorlar.loc[i, 'experience_company'] = doctor_details['experience_company']
            doktorlar.loc[i, 'application_ld_json'] = doctor_details['application_ld_json']

            print(f"Profile {i} details added to the dataframe")

    except Exception as e:
        print(f"Error: {e}")
        error_indices.append(i)

In [33]:
start_time = time.time()
async with aiohttp.ClientSession() as session:
    tasks = []
    for i in range(13538, len(doktorlar)):
        url = doktorlar["profile"][i]
        tasks.append(get_profile_details(url, session, i))
    await asyncio.gather(*tasks)
end_time = time.time()

print(f"Time taken to get the details of profiles: {end_time - start_time} seconds")

Profile 13539 details added to the dataframe
Profile 13558 details added to the dataframe
Profile 13561 details added to the dataframe
Profile 13555 details added to the dataframe
Profile 13538 details added to the dataframe
Profile 13548 details added to the dataframe
Profile 13565 details added to the dataframe
Profile 13618 details added to the dataframe
Profile 13622 details added to the dataframe
Profile 13626 details added to the dataframe
Profile 13608 details added to the dataframe
Profile 13556 details added to the dataframe
Profile 13562 details added to the dataframe
Profile 13578 details added to the dataframe
Profile 13574 details added to the dataframe
Profile 13587 details added to the dataframe
Profile 13580 details added to the dataframe
Profile 13569 details added to the dataframe
Profile 13640 details added to the dataframe
Profile 13573 details added to the dataframe
Profile 13568 details added to the dataframe
Profile 13598 details added to the dataframe
Profile 13

In [43]:
# get the error indices by checking data_id column in the doktorlar dataframe
error_indices = doktorlar[doktorlar["data_id"].isnull()].index.tolist()
print(len(error_indices))

# call the function again for the error indices
start_time = time.time()
async with aiohttp.ClientSession() as session:
    tasks = []
    for i in error_indices:
        url = doktorlar["profile"][i]
        tasks.append(get_profile_details(url, session, i))
    await asyncio.gather(*tasks)
end_time = time.time()

print(f"Time taken to get the details of profiles: {end_time - start_time} seconds")

70
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not subscriptable
Error: 'NoneType' object is not

In [44]:
# save the doktorlar dataframe to a csv file
doktorlar.to_csv("doktorlar_details.csv", index=False)

In [45]:
doktorlar.head()

,id,image_link,title,name,is_verified,profile,gender,location,speciality,data_id,expert_point,services,about_content,eed_items,expert_interests,experience_company,application_ld_json
0,1,https://www.doktorsitesi.com/media/cache/profi...,Psk.,Arzu Kantarcıoğlu,NaN,https://www.doktorsitesi.com/psk-arzu-kantarci...,female,bursa,psikoloji,10344,None,[],,[],[],None,"[{\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t""@context""..."
1,2,https://www.doktorsitesi.com/media/cache/profi...,Dr.,Abdullah Atasoy,NaN,https://www.doktorsitesi.com/dr-abdullah-ataso...,male,balikesir,aile-hekimligi,10953,None,[],,[],[],None,"[{\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t""@context""..."
2,3,https://www.doktorsitesi.com/media/cache/profi...,Dr.,Atakan Saçlı,NaN,https://www.doktorsitesi.com/dr-atakan-sacli/p...,male,izmir,pratisyen-hekimlik,65885,None,[],,[],"[""Bel F\u0131t\u0131\u011f\u0131"", ""Sch\u00fcs...",None,"[{\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t""@context""..."
3,4,https://www.doktorsitesi.com/media/cache/profi...,Op. Dr.,Ayla Özer(ağar),NaN,https://www.doktorsitesi.com/op-dr-ayla-ozerag...,female,izmir,kadin-hastaliklari-ve-dogum,180072,None,[],,"[""Diploma Ald\u0131\u011f\u0131 Kurum\nEGE \u0...","[""Adet A\u011fr\u0131lar\u0131 (Dismenore)"", ""...",None,"[{\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t""@context""..."
4,5,https://www.doktorsitesi.com/media/cache/profi...,Op. Dr.,Ayşe Gülgün Bilgin,NaN,https://www.doktorsitesi.com/op-dr-ayse-gulgun...,female,mugla,kadin-hastaliklari-ve-dogum,180087,None,[],,"[""Diploma Ald\u0131\u011f\u0131 Kurum\nEGE \u0...","[""Plasenta Previa"", ""K\u00fcrtaj"", ""Adet D\u00...",None,"[{\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t""@context""..."


In [46]:
# copy id, data_id, application_ld_json columns to a new dataframe
doktorlar_data_id = doktorlar[["id", "data_id", "application_ld_json"]]

In [48]:
doktorlar_data_id.to_csv("doktorlar_data_id.csv", index=False)
doktorlar_data_id.head()

,id,data_id,application_ld_json
0,1,10344,"[{\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t""@context""..."
1,2,10953,"[{\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t""@context""..."
2,3,65885,"[{\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t""@context""..."
3,4,180072,"[{\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t""@context""..."
4,5,180087,"[{\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t""@context""..."


In [52]:
# remove application_ld_json column from the doktorlar dataframe
# doktorlar.drop(columns=["application_ld_json"], inplace=True)
doktorlar.to_csv("doktorlar_details.csv", index=False, encoding="utf-8")
doktorlar.head()

,id,image_link,title,name,is_verified,profile,gender,location,speciality,data_id,expert_point,services,about_content,eed_items,expert_interests,experience_company
0,1,https://www.doktorsitesi.com/media/cache/profi...,Psk.,Arzu Kantarcıoğlu,NaN,https://www.doktorsitesi.com/psk-arzu-kantarci...,female,bursa,psikoloji,10344,None,[],,[],[],None
1,2,https://www.doktorsitesi.com/media/cache/profi...,Dr.,Abdullah Atasoy,NaN,https://www.doktorsitesi.com/dr-abdullah-ataso...,male,balikesir,aile-hekimligi,10953,None,[],,[],[],None
2,3,https://www.doktorsitesi.com/media/cache/profi...,Dr.,Atakan Saçlı,NaN,https://www.doktorsitesi.com/dr-atakan-sacli/p...,male,izmir,pratisyen-hekimlik,65885,None,[],,[],"[""Bel F\u0131t\u0131\u011f\u0131"", ""Sch\u00fcs...",None
3,4,https://www.doktorsitesi.com/media/cache/profi...,Op. Dr.,Ayla Özer(ağar),NaN,https://www.doktorsitesi.com/op-dr-ayla-ozerag...,female,izmir,kadin-hastaliklari-ve-dogum,180072,None,[],,"[""Diploma Ald\u0131\u011f\u0131 Kurum\nEGE \u0...","[""Adet A\u011fr\u0131lar\u0131 (Dismenore)"", ""...",None
4,5,https://www.doktorsitesi.com/media/cache/profi...,Op. Dr.,Ayşe Gülgün Bilgin,NaN,https://www.doktorsitesi.com/op-dr-ayse-gulgun...,female,mugla,kadin-hastaliklari-ve-dogum,180087,None,[],,"[""Diploma Ald\u0131\u011f\u0131 Kurum\nEGE \u0...","[""Plasenta Previa"", ""K\u00fcrtaj"", ""Adet D\u00...",None


In [53]:
# fix unicode characters in the dataframe and save it to a csv file
doktorlar = pd.read_csv("doktorlar_details.csv", encoding="utf-8")
doktorlar.to_csv("doktorlar_details.csv", index=False, encoding="utf-8")